In [2]:
import numpy as np 
import pandas as pd 
import csv
from sklearn.model_selection import train_test_split

snp_data = pd.read_csv('output.csv')
train_data = pd.read_csv('STR_train.csv',sep=';')



<font color='green'>Data Preprocessing.</font> SNP из FinalReport.csv и длины STR из STR_train.csv объединяются таким образом, что каждая строка в наших данных имеет один уникальный animal_id, 24 длины STR, и все уникальны SNP в FinalReport.csv

In [9]:
all_ids = snp_data['animal_id'].unique()
all_snp = snp_data['SNP Name'].unique()
all_str = train_data['STR Name'].unique()

x=[]
x = np.insert(all_snp,0,'animal_id')
for str_n in all_str:
    x = np.append(x, str_n+'_allele1')
    x = np.append(x, str_n+'_allele2')
    
finaldf = pd.DataFrame(x)
finaldf = finaldf.T
finaldf.columns = finaldf.iloc[0]


Данные SNP из FinalReport.csv кодируются по следующей схеме: <br>
AA = 0
AB = 1
BB = 2
</br>
При отсутствии данных они заполняются с «reference» или 0.

In [4]:
gozi = finaldf
rowIndex = gozi.index[0]
for animal_id in all_ids:
    rowsnp = snp_data.loc[snp_data['animal_id'] == animal_id]
    rowstr = train_data.loc[train_data['animal_id'] == animal_id]
    rowIndex = rowIndex + 1
    gozi.loc[rowIndex, 'animal_id'] = animal_id
    for row in rowsnp.itertuples():
        if row._6=='A' and row._7=='A':
            gozi.at[rowIndex, str(row._3)] = 0
        elif (row._6=='A' and row._7=='B') or (row._6=='B' and row._7=='A') :
            gozi.at[rowIndex, str(row._3)] = 1
        elif row._6=='B' and row._7=='B':
            gozi.at[rowIndex, str(row._3)] = 2
        else:
            gozi.at[rowIndex, str(row._3)] = 0
    for row in rowstr.itertuples():
        gozi.at[rowIndex, str(row._2)+'_allele1'] = row.Allele1
        gozi.at[rowIndex, str(row._2)+'_allele2'] = row.Allele2
#gozi.to_csv('gozi_missing_0.csv')

Есть ~535 animal_ids, для которых отсутствует большая часть длины ST row index 1967-2501. Эти строки удалены из наших данных. Есть около 100 столбцов между index 2887 и 2987, для которых более половины animal_ids не имеют данных. Эти столбцы были удалены, как и 3 других столбца (BovineHD0200....), для которых также отсутствует более половины записей. Все они удалены из наших данных.

In [52]:
file = pd.read_csv('gozi_missing_0.csv',low_memory=False)
file = file.drop(gzv.iloc[:, 2887:2987], axis=1)
file = file.drop(gzz.index[1967:2501])
file.drop(labels='BovineHD0200037029',axis=1,inplace=True)
file.drop(labels='BovineHD0300010318',axis=1,inplace=True)
file.drop(labels='BovineHD0300010328',axis=1,inplace=True)
file.to_csv('no_missing_snps.csv')



 пропущенные значения длины STR аллеля были заполнены медианным значением этого столбца. Я сделал это в Excel. <https://drive.google.com/file/d/1BAGedFXgS95m-pbM_D7AYyXnojPxX24n/view?usp=sharing> - это входной файл, используемый в следующем разделе.

Read data, and separate them into X (inputs) and y (labels)

In [3]:
all_data = pd.read_csv('snp_0_str_median.csv',low_memory=False)
x_data = all_data.iloc[:, :2883] # First 2880 columns are SNPs
y_data = all_data.iloc[:, -24:]  # Remaining columns are STR lengths

Поскольку наши данные в высокой степени «категоричны» — значения данных SNP могут принимать только дискретные значения, которые представляют категории. Кроме того, для каждого конкретного STR длина принимает значения в небольшом диапазоне. Для таких данных два основных варианта моделей ИИ — это либо нейронная сеть со слоем Embedding, либо модель градиентного усиления. Модели градиентного усиления, такие как XGBoost, показали очень хорошую производительность для высокоразмерных категориальных данных. Я пробовал использовать эти данные в глубокой нейронной сети с внедрением. Модель XGBoost дает лучшие результаты.

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import xgboost as xgb



X = x_data  
y = y_data 

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create DMatrix for XGBoost
dtrain = xgb.DMatrix(X_train_scaled, label=y_train)
dtest = xgb.DMatrix(X_test_scaled, label=y_test)

# Set XGBoost parameters
params = {
    'max_depth': 6,
    'eta': 0.3,
    'objective': 'reg:squarederror',
    'num_boost_round': 100,
    'early_stopping_rounds': 10,
    'eval_metric': 'rmse'
}

# Train the model
model = xgb.train(params, dtrain, num_boost_round=1000,
                  evals=[(dtrain, 'train'), (dtest, 'test')],
                  early_stopping_rounds=10, verbose_eval=100)

# Make predictions
y_pred = model.predict(dtest)
pd.DataFrame(y_pred).to_csv('pred.csv')
X_test.to_csv('xt.csv')
# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared Score: {r2}")

# Feature importance
importance = model.get_score(importance_type='gain')
importance = sorted(importance.items(), key=lambda x: x[1], reverse=True)
print("Top 10 most important features:")
for feature, score in importance[:10]:
    print(f"{feature}: {score}")

# Save the model
#model.save_model('str_length_prediction_model.json')

#print("Model saved as 'str_length_prediction_model.json'")

F:\az\envs\tf\lib\site-packages\xgboost\core.py:158: UserWarning: [06:09:27] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "early_stopping_rounds", "num_boost_round" } are not used.

  warnings.warn(smsg, UserWarning)


[0]	train-rmse:36.85756	test-rmse:37.31440
[23]	train-rmse:1.02799	test-rmse:6.99556
Mean Squared Error: 48.95500730545925
R-squared Score: 0.45429691672325134
Top 10 most important features:
f63: 51586.484375
f2235: 26947.2734375
f1706: 25844.921875
f1998: 20436.478515625
f1593: 19988.34375
f1713: 16088.919921875
f1792: 14964.7666015625
f2242: 12096.85546875
f2217: 11740.6435546875
f1788: 11443.03125


Как мы видим, модель XGBoost может быть использована для поиска признаков (SNP), которые вносят наибольший вклад в правильные прогнозы. План состоял в том, чтобы использовать 300 лучших признаков для создания новой модели, которая просто использует эти 300 признаков в качестве входных данных. Но затем я понял, что это не та задача

создаем файл с SNP и их прогнозируемыми длинами STR. Его можно посмотреть здесь: <https://drive.google.com/file/d/1iA3gzxcAGKxPgf-WH2KmqL1dUEwV5-9i/view?usp=sharing>

In [8]:
file =  np.concatenate( (X_test, y_pred), axis=1 ) 
pd.DataFrame(g).to_csv('predictions.csv')